# Lecture 12 Visualizing and Understanding

**Gradient Ascent** : Generate a synthetic image that maximally activates a neuron

$$ arg max_{I} S_c(I) - \lambda ||I||_2^2 $$

**Deep Dream** : Amplify existing features

In [ ]:
def objective_L2(det):
   det.diff[:] = det.data
  
def make_step(net, step_size = 1.5, end = 'inception_4c/output', 
              jitter = 32, clip = True, objective = objective_L2):
  arc = net.blobs['data'] # input image is stored in net's 'data' blob
  det = net.blobs[end]

  ox, oy = np.random.randint(-jitter, jitter+1, 2)
  arc.data[0] = np.roll(np.roll(arc.data[0], ox, -1), oy, -2) # apply jitter shift

  net.forward(end = end)
  objective(det)  # specify the optimization objective
  net.backward(start = end)
  q = arc.diff[0]

  # apply normailzed ascent step to the input image
  arc.data[:] += np.rall(np.roll(arc.data[0], -ox, -1), -oy, -2) # unshift image

  if clip:
    bias = net.transformer.mean['data']
    arc.data[:] = np.clip(arc, data, -bias, 255-bias)

**Gram Matrix** 

Each Layer of CNN gives C x H x W tensor of features; HxW grid of C-dimensional vectors
Outer product of two C-dimensional vectors gives C x C matrix measuring co-occurence
-> Average over all HW pairs of vectors giving "Gram Matrix"  of shape C x C

- Efficient to compute
- Nice to Texture

**Neural Texture Synthesis**
1. Pretrain a CNN on ImageNet(VGG-19)
2. Run input texture forward through CNN, record activations on every layer; layer i gives feature map of shape C_i x H_i x W_i
3. At each layer compute the *Gram Matrix* giving outer product of features:
$$ G_{ij}^l = \sum_{k}F_{ik}^lF_{jk}^l$$
4. Initialize generated image from random noise
5. Pass generated image through CNN, compute Gram matrix on each layer
6. Compute loss: weighted sum of L2 distance between Gram matrices
7. Backprop to get gradient on image
8. Make gradient step on image
9. GOTO 5
